In [3]:
from datasets import load_dataset
from datasets import load_metric
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from functools import partial
from datasets import load_metric
import torch
from lab5_helper import collate_fn, get_train_features, predict
from lab5_train import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

lr = 2e-5
n_epochs = 3
weight_decay = 0.01
warmup_steps = 200
bert_map = {
    'bengali': 'xlm-roberta-base', 
    'english': 'xlm-roberta-base', 
    'indonesian': 'xlm-roberta-base', 
    'arabic': 'xlm-roberta-base'
}
device = 'cpu'

compute_squad = load_metric("squad_v2")
dataset = load_dataset("copenlu/answerable_tydiqa")

for split in dataset.keys():
    dataset[split] = dataset[split].add_column('id', list(range(len(dataset[split]))))

language = 'bengali'
language2 = 'indonesian'

model = torch.load(f'{language}_xlm-roberta-base_classification.pt')
language_dataset = dataset.filter(lambda example: example['language'] == language2)
tk = AutoTokenizer.from_pretrained(bert_map[language2], max_len=300)

questions = dataset['validation'][0:1]['question_text']
documents = dataset['validation'][0:1]['document_plaintext']

inputs = tk(questions, documents, return_tensors="pt", truncation=True, padding=True, max_length=300)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.